In [1]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
import urllib.request

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2021"

In [3]:
source = urllib.request.urlopen(link).read()
soup = bs.BeautifulSoup(source,'lxml')

In [4]:
tables = soup.find_all('table',class_='wikitable sortable')

In [5]:
len(tables)

4

In [6]:
df1 = pd.read_html(str(tables[0]))[0]
df2 = pd.read_html(str(tables[1]))[0]
df3 = pd.read_html(str(tables[2]))[0]
df4 = pd.read_html(str(tables[3]).replace("'1\"\'",'"1"'))[0]

In [7]:
df = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [8]:
df

,Opening,Opening.1,Title,Production company,Cast and crew,.mw-parser-output .tooltip-dotted{border-bottom:1px dotted;cursor:help}Ref.,Ref.
0,JANUARY,1,Shadow in the Cloud,Vertical Entertainment,Roseanne Liang (director/screenplay); Max Land...,[2],NaN
1,JANUARY,13,The White Tiger,Netflix,Ramin Bahrani (director/screenplay); Adarsh Go...,NaN,NaN
2,JANUARY,14,Locked Down,HBO Max / Warner Bros. Pictures,Doug Liman (director); Steven Knight (screenpl...,[3],NaN
3,JANUARY,15,The Dig,Netflix / Clerkenwell Films,Simon Stone (director); Moira Buffini (screenp...,[4],NaN
4,JANUARY,15,Outside the Wire,Netflix,"Mikael Håfström (director); Rob Yescombe, Rowa...",[5],NaN
...,...,...,...,...,...,...,...
306,DECEMBER,22,The Matrix: Resurrections,HBO Max / Warner Bros. Pictures / Village Road...,Lana Wachowski (director/screenplay); Aleksand...,NaN,[231]
307,DECEMBER,22,Sing 2,Universal Pictures / Illumination,Garth Jennings (director/screenplay); Matthew ...,NaN,[232]
308,DECEMBER,22,The King's Man,20th Century Studios / Marv Studios,Matthew Vaughn (director/screenplay); Karl Gaj...,NaN,[112]
309,DECEMBER,25,American Underdog: The Kurt Warner Story,Lionsgate,"Erwin brothers (directors); David Aaron, Jon E...",NaN,NaN


In [9]:
df_2021 = df[['Title','Cast and crew']]

In [10]:
df_2021

,Title,Cast and crew
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...
3,The Dig,Simon Stone (director); Moira Buffini (screenp...
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa..."
...,...,...
306,The Matrix: Resurrections,Lana Wachowski (director/screenplay); Aleksand...
307,Sing 2,Garth Jennings (director/screenplay); Matthew ...
308,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...
309,American Underdog: The Kurt Warner Story,"Erwin brothers (directors); David Aaron, Jon E..."


In [11]:
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
tmdb.api_key = 'b6e2dbc1c5fe379b61bd7a1810e7b203'

In [12]:
from tmdbv3api import Movie
tmdb_movie = Movie() 
def get_genre(x):
    genres = []
    result = tmdb_movie.search(x)
    if not result:
      return np.NaN
    else:
      movie_id = result[0].id
      response = requests.get('https://api.themoviedb.org/3/movie/{}?api_key={}'.format(movie_id,tmdb.api_key))
      data_json = response.json()
      if data_json['genres']:
          genre_str = " " 
          for i in range(0,len(data_json['genres'])):
              genres.append(data_json['genres'][i]['name'])
          return genre_str.join(genres)
      else:
          return np.NaN

In [13]:
df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))

<ipython-input-13-252c1cda1e49>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['genres'] = df_2021['Title'].map(lambda x: get_genre(str(x)))


In [14]:
df_2021

,Title,Cast and crew,genres
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Crime Drama
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction
...,...,...,...
306,The Matrix: Resurrections,Lana Wachowski (director/screenplay); Aleksand...,Science Fiction Action Adventure
307,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Family Comedy Animation Drama Music
308,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,War Action Thriller Adventure
309,American Underdog: The Kurt Warner Story,"Erwin brothers (directors); David Aaron, Jon E...",Drama


In [15]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [16]:
df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-16-059d9cfe3ca6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['director_name'] = df_2021['Cast and crew'].map(lambda x: get_director(str(x)))


In [17]:
def get_actor1(x):
    return ((x.split("screenplay); ")[-1]).split(", ")[0])

In [18]:
df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))

<ipython-input-18-fbf9a4980306>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_1_name'] = df_2021['Cast and crew'].map(lambda x: get_actor1(str(x)))


In [19]:
def get_actor2(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[1])

In [20]:
df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))

<ipython-input-20-597eaf6de001>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2021['actor_2_name'] = df_2021['Cast and crew'].map(lambda x: get_actor2(str(x)))


In [21]:
def get_actor3(x):
    if len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    else:
        return ((x.split("screenplay); ")[-1]).split(", ")[2])

In [22]:
df_2021['actor_3_name'] = df_2021['Cast and crew'].map(lambda x: get_actor3(str(x)))

In [24]:
df_2021

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Shadow in the Cloud,Roseanne Liang (director/screenplay); Max Land...,Horror Action War,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale
1,The White Tiger,Ramin Bahrani (director/screenplay); Adarsh Go...,Crime Drama,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas
2,Locked Down,Doug Liman (director); Steven Knight (screenpl...,Comedy Crime Drama,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant
3,The Dig,Simon Stone (director); Moira Buffini (screenp...,Drama History,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James
4,Outside the Wire,"Mikael Håfström (director); Rob Yescombe, Rowa...",Thriller Action Science Fiction,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham
...,...,...,...,...,...,...,...
306,The Matrix: Resurrections,Lana Wachowski (director/screenplay); Aleksand...,Science Fiction Action Adventure,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II
307,Sing 2,Garth Jennings (director/screenplay); Matthew ...,Family Comedy Animation Drama Music,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson
308,The King's Man,Matthew Vaughn (director/screenplay); Karl Gaj...,War Action Thriller Adventure,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans
309,American Underdog: The Kurt Warner Story,"Erwin brothers (directors); David Aaron, Jon E...",Drama,Erwin brothers,Zachary Levi,Anna Paquin,Ser'Darius Blain


In [25]:
df_2021 = df_2021.rename(columns={'Title':'movie_title'})

In [26]:
new_df21 = df_2021.loc[:,['director_name','actor_1_name','actor_2_name','actor_3_name','genres','movie_title']]

In [27]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,The White Tiger
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire
...,...,...,...,...,...,...
306,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,Science Fiction Action Adventure,The Matrix: Resurrections
307,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,Sing 2
308,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,The King's Man
309,Erwin brothers,Zachary Levi,Anna Paquin,Ser'Darius Blain,Drama,American Underdog: The Kurt Warner Story


In [28]:
new_df21['comb'] = new_df21['actor_1_name'] + ' ' + new_df21['actor_2_name'] + ' '+ new_df21['actor_3_name'] + ' '+ new_df21['director_name'] +' ' + new_df21['genres']

In [29]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,Shadow in the Cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,The White Tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,Locked Down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,The Dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,Outside the Wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
306,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,Science Fiction Action Adventure,The Matrix: Resurrections,Keanu Reeves Carrie-Anne Moss Yahya Abdul-Mate...
307,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,Sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
308,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,The King's Man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
309,Erwin brothers,Zachary Levi,Anna Paquin,Ser'Darius Blain,Drama,American Underdog: The Kurt Warner Story,Zachary Levi Anna Paquin Ser'Darius Blain Erwi...


In [30]:
new_df21.isna().sum()

director_name     0
actor_1_name      0
actor_2_name      6
actor_3_name     19
genres            1
movie_title       0
comb             20
dtype: int64

In [31]:
new_df21 = new_df21.dropna(how='any')

In [32]:
new_df21.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [33]:
new_df21['movie_title'] = new_df21['movie_title'].str.lower()

<ipython-input-33-6e765124d575>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df21['movie_title'] = new_df21['movie_title'].str.lower()


In [34]:
new_df21

,director_name,actor_1_name,actor_2_name,actor_3_name,genres,movie_title,comb
0,Roseanne Liang,Chloë Grace Moretz,Taylor John Smith,Beulah Koale,Horror Action War,shadow in the cloud,Chloë Grace Moretz Taylor John Smith Beulah Ko...
1,Ramin Bahrani,Adarsh Gourav,Rajkummar Rao,Priyanka Chopra Jonas,Crime Drama,the white tiger,Adarsh Gourav Rajkummar Rao Priyanka Chopra Jo...
2,Doug Liman,Anne Hathaway,Chiwetel Ejiofor,Stephen Merchant,Comedy Crime Drama,locked down,Anne Hathaway Chiwetel Ejiofor Stephen Merchan...
3,Simon Stone,Carey Mulligan,Ralph Fiennes,Lily James,Drama History,the dig,Carey Mulligan Ralph Fiennes Lily James Simon ...
4,Mikael Håfström,Anthony Mackie,Damson Idris,Emily Beecham,Thriller Action Science Fiction,outside the wire,Anthony Mackie Damson Idris Emily Beecham Mika...
...,...,...,...,...,...,...,...
306,Lana Wachowski,Keanu Reeves,Carrie-Anne Moss,Yahya Abdul-Mateen II,Science Fiction Action Adventure,the matrix: resurrections,Keanu Reeves Carrie-Anne Moss Yahya Abdul-Mate...
307,Garth Jennings,Matthew McConaughey,Reese Witherspoon,Scarlett Johansson,Family Comedy Animation Drama Music,sing 2,Matthew McConaughey Reese Witherspoon Scarlett...
308,Matthew Vaughn,Ralph Fiennes,Gemma Arterton,Rhys Ifans,War Action Thriller Adventure,the king's man,Ralph Fiennes Gemma Arterton Rhys Ifans Matthe...
309,Erwin brothers,Zachary Levi,Anna Paquin,Ser'Darius Blain,Drama,american underdog: the kurt warner story,Zachary Levi Anna Paquin Ser'Darius Blain Erwi...


In [35]:
old_df = pd.read_csv('main_data.csv')

In [36]:
final_df = old_df.append(new_df21,ignore_index=True)

In [37]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [38]:
final_df.to_csv('complete_data.csv',index=False)